# Uso de archivo zip 
uso de zip para descomprimir y obtener los datos

In [36]:
import h5py
import cv2
import numpy as np
import os
import zipfile
from matplotlib import image

files=zipfile.ZipFile('cats_and_dogs_small.zip','r')
files.extractall('')

x_dog=[]
x_cat=[]



# Filtrado de datos en gatos y 

In [37]:
from PIL import Image
x_size=150
y_size=150

for name in files.namelist():
    if '/dogs/' in name and '.jpg' in name:
        a=cv2.imread(name)
        a=cv2.resize(a,(x_size,y_size)) # Dimensión de la imagen
        img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        #img2=img.resize(200,200) # Mobilenet (224,224,3)
        x_dog.append(img)
        
    elif '/cats/' in name and '.jpg' in name:
        a=cv2.imread(name)
        a=cv2.resize(a,(x_size,y_size)) # Dimensión de la imagen
        img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        x_cat.append(img)
print(len(x_dog),len(x_cat))
x_dog=np.stack(x_dog,axis=0)
x_cat=np.stack(x_cat,axis=0)

2000 2000


In [38]:
"""
a=cv2.imread(name)
a=cv2.resize(a,(200,200))
print(a.shape)
img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
print(img.shape)
"""

'\na=cv2.imread(name)\na=cv2.resize(a,(200,200))\nprint(a.shape)\nimg = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)\nprint(img.shape)\n'

# Normalización de datos

In [39]:

print(type(x_dog),x_dog.shape)
print(type(x_cat),x_cat.shape)
x_dog=x_dog.astype('float32')
x,y,z,w=x_dog.shape
y_dog=np.zeros((x,1),dtype=int)
x_cat=x_cat.astype('float32')
x,y,z,w=x_cat.shape
y_cat=np.ones((x,1),dtype=int)
#x_dog=(x_dog/127.5)-1#x_dog/=255
#x_cat=(x_cat/127.5)-1#x_cat/=255
## Conjunto combinado de perros y gatos
x_comb=np.vstack((x_dog,x_cat))
y_comb=np.vstack((y_dog,y_cat))
print(x_comb.ndim,x_comb.shape)
print(y_dog)

<class 'numpy.ndarray'> (2000, 150, 150, 3)
<class 'numpy.ndarray'> (2000, 150, 150, 3)
4 (4000, 150, 150, 3)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [40]:

### ONE HOT
from tensorflow.keras.utils import to_categorical
y_dog_oh=to_categorical(y_dog,y_dog.max()+2)
y_cat_oh=to_categorical(y_cat,y_cat.max()+1)
print(y_comb[3455])
y_comb_oh=to_categorical(y_comb,y_comb.max()+1)
print(y_comb_oh[3455])
print(type(y_comb_oh),y_comb_oh.shape)
print(y_cat_oh.shape)
print(y_dog_oh.shape)
print(y_dog_oh)
#print(y_cat_oh)

[1]
[0. 1.]
<class 'numpy.ndarray'> (4000, 2)
(2000, 2)
(2000, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


# División 60 20 20

In [41]:
xx,yy,ww,zz=x_cat.shape
x_train=np.vstack((x_cat[:int(xx*0.6),:,:,:],x_dog[:int(xx*0.6),:,:,:]))
y_train=np.vstack((y_cat_oh[:int(xx*0.6),:],y_dog_oh[:int(xx*0.6),:]))
x_val=np.vstack((x_cat[int(xx*0.6):int(xx*0.8),:,:,:],x_dog[int(xx*0.6):int(xx*0.8),:,:,:]))
y_val=np.vstack((y_cat_oh[int(xx*0.6):int(xx*0.8),:],y_dog_oh[int(xx*0.6):int(xx*0.8),:]))
x_test=np.vstack((x_cat[int(xx*0.8):,:,:,:],x_dog[int(xx*0.8):,:,:,:]))
y_test=np.vstack((y_cat_oh[int(xx*0.8):,:],y_dog_oh[int(xx*0.8):,:]))

print(x_test.min())

0.0


# Red LeNet5

## preprocessing y deshabilitar entrenamiento


In [42]:
"""
import tensorflow as tf
from keras.models import Model, load_model
#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint =  ModelCheckpoint('vgg16_finetune.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True) 

x_train=np.vstack((x_cat[:int(xx*0.6),:,:,1:2],x_dog[:int(xx*0.6),:,:,1:2]))
y_train=np.vstack((y_cat_oh[:int(xx*0.6),:],y_dog_oh[:int(xx*0.6),:]))
x_val=np.vstack((x_cat[int(xx*0.6):int(xx*0.8),:,:,1:2],x_dog[int(xx*0.6):int(xx*0.8),:,:,1:2]))
y_val=np.vstack((y_cat_oh[int(xx*0.6):int(xx*0.8),:],y_dog_oh[int(xx*0.6):int(xx*0.8),:]))
x_test=np.vstack((x_cat[int(xx*0.8):,:,:,1:2],x_dog[int(xx*0.8):,:,:,1:2]))
y_test=np.vstack((y_cat_oh[int(xx*0.8):,:],y_dog_oh[int(xx*0.8):,:]))


model=Sequential()
model.add(tf.keras.layers.Conv2D(6,(5,5),input_shape=(x_size,y_size,1),activation='tanh',padding='valid',strides=1)) #C1

model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2))) #S2

model.add(tf.keras.layers.Conv2D(16,(5,5),activation='tanh',padding='valid',strides=1)) #c3

model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2))) #s4
model.add(tf.keras.layers.Flatten())

model.add(Dense(120,activation='tanh')) #c5
model.add(Dense(84,activation='tanh')) #c6

from keras.layers import Layer
from keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)
    
model.add(RBFLayer(2,0.5)) #c7

model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.SGD(learning_rate=0.25),metrics=['accuracy'])
model.fit(x_train,y_train,verbose=1, batch_size=64,epochs=100,validation_data=(x_val,y_val))
"""

"\nimport tensorflow as tf\nfrom keras.models import Model, load_model\n#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense,Flatten,Dropout,Input\nfrom tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau\n\nlr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)\ncheckpoint =  ModelCheckpoint('vgg16_finetune.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)\nearlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True) \n\nx_train=np.vstack((x_cat[:int(xx*0.6),:,:,1:2],x_dog[:int(xx*0.6),:,:,1:2]))\ny_train=np.vstack((y_cat_oh[:int(xx*0.6),:],y_dog_oh[:int(xx*0.6),:]))\nx_val=np.vstack((x_cat[int(xx*0.6):int(xx*0.8),:,:,1:2],x_dog[int(xx*0.6):int(xx*0.8),:,:,1:2]))\ny_val=np.vstack((y

In [43]:
"""
pred=model.predict(x_test)
pred=np.argmax(pred,axis=1)
y1=np.argmax(y_test,axis=1)

#label=np.argmax(yp_oh)
exactitud_test=0
for a in range(len(pred)):
    if pred[a]==y1[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')
"""

"\npred=model.predict(x_test)\npred=np.argmax(pred,axis=1)\ny1=np.argmax(y_test,axis=1)\n\n#label=np.argmax(yp_oh)\nexactitud_test=0\nfor a in range(len(pred)):\n    if pred[a]==y1[a]:\n        exactitud_test+=1\nprint('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')\n"

   # Red ResNet50

In [48]:
import tensorflow as tf
from keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint =  ModelCheckpoint('resnet50.h', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)    
x_train_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_train))
x_val_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_val))
x_test_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_test))
model = ResNet50(input_shape=(x_size,y_size,3),weights=None,include_top=False) ## Colocar otro top
for i in range(165):
    model.layers[i].trainable=False
#model.summary()
sal=model.output
sal=Flatten()(sal)
sal=Dense(502,activation='selu')(sal)
sal=Dropout(0.26)(sal)
sal = Dense(256, activation='relu')(sal)
sal=Dense(100,activation='selu')(sal)
sal=Dense(50,activation='relu')(sal)
sal = Dense(2, activation='softmax')(sal)
#Se unen la CNN y el top
resnet50_custom=Model(inputs=model.input, outputs=sal)
resnet50_custom.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])
history=resnet50_custom.fit(x_train_res,y_train,batch_size=64,epochs=10, validation_data=(x_val_res,y_val),callbacks=[lr_reduce,earlystopper,checkpoint])
#model.summary()
#preds=model.predict(x_comb)
#model.add(tf.keras.layers.Flaten())
"""
tf.keras.applications.MobileNet(
    input_shape=(200,200,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    **kwargs
)
"""

Epoch 1/10
38/38 [==============================] - 57s 1s/step - loss: 1.3805 - accuracy: 0.4900 - val_loss: 0.7172 - val_accuracy: 0.5075

Epoch 00001: val_accuracy improved from -inf to 0.50750, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
38/38 [==============================] - 51s 1s/step - loss: 0.7683 - accuracy: 0.5046 - val_loss: 0.7103 - val_accuracy: 0.5100

Epoch 00002: val_accuracy improved from 0.50750 to 0.51000, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 3/10
38/38 [==============================] - 52s 1s/step - loss: 0.7437 - accuracy: 0.5113 - val_loss: 0.7002 - val_accuracy: 0.5013

Epoch 00003: val_accuracy did not improve from 0.51000
Epoch 4/10
38/38 [==============================] - 53s 1s/step - loss: 0.7219 - accuracy: 0.5258 - val_loss: 0.6868 - val_accuracy: 0.5512

Epoch 00004: val_accuracy improved from 0.51000 to 0.55125, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


...

Epoch 5/10
38/38 [==============================] - 51s 1s/step - loss: 0.7222 - accuracy: 0.5179 - val_loss: 0.6867 - val_accuracy: 0.5375

Epoch 00005: val_accuracy did not improve from 0.55125
Epoch 6/10
38/38 [==============================] - 53s 1s/step - loss: 0.7098 - accuracy: 0.5192 - val_loss: 0.6827 - val_accuracy: 0.5437

Epoch 00006: val_accuracy did not improve from 0.55125
Epoch 7/10
38/38 [==============================] - 54s 1s/step - loss: 0.7025 - accuracy: 0.5242 - val_loss: 0.6769 - val_accuracy: 0.5875

Epoch 00007: val_accuracy improved from 0.55125 to 0.58750, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 8/10
38/38 [==============================] - 52s 1s/step - loss: 0.6857 - accuracy: 0.5558 - val_loss: 0.6722 - val_accuracy: 0.5838

Epoch 00008: val_accuracy did not improve from 0.58750
Epoch 9/10
38/38 [==============================] - 53s 1s/step - loss: 0.6897 - accuracy: 0.5617 - val_loss: 0.6840 - val_accuracy: 0.5437

Epoch 00009: val_accuracy did not improve from 0.58750
Epoch 10/10
38/38 [==============================] - 53s 1s/step - loss: 0.6804 - accuracy: 0.5771 - val_loss: 0.6654 - val_accuracy: 0.6200

Epoch 00010: val_accuracy improved from 0.58750 to 0.62000, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


'\ntf.keras.applications.MobileNet(\n    input_shape=(200,200,3),\n    alpha=1.0,\n    depth_multiplier=1,\n    dropout=0.001,\n    include_top=True,\n    weights="imagenet",\n    input_tensor=None,\n    pooling=None,\n    classes=1000,\n    classifier_activation="softmax",\n    **kwargs\n)\n'

In [49]:
#preds2=np.argmax(preds,axis=1)
## Sale 111
pred=resnet50_custom.predict(x_test_res)
pred=np.argmax(pred,axis=1)
y1=np.argmax(y_test,axis=1)

#label=np.argmax(yp_oh)
exactitud_test=0
for a in range(len(pred)):
    if pred[a]==y1[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')

exactitud de la prueba=  55.5 %


# EfficientNetB0

In [53]:
import tensorflow as tf
from keras.models import Model, load_model
#from tensorflow.keras.applications.efficientnet.EfficientNetB0 import EfficientNetB0
#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint =  ModelCheckpoint('resnet50.h', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)    

x_train_res=tf.keras.applications.efficientnet.preprocess_input(np.copy(x_train))
x_val_res=tf.keras.applications.efficientnet.preprocess_input(np.copy(x_val))
x_test_res=tf.keras.applications.efficientnet.preprocess_input(np.copy(x_test))

model = tf.keras.applications.efficientnet.EfficientNetB0(input_shape=(x_size,y_size,3),weights=None,include_top=False) ## Colocar otro top
#model.summary()
sal=model.output
sal=Flatten()(sal)
sal=Dense(502,activation='relu')(sal)
sal=Dropout(0.26)(sal)
sal = Dense(256, activation='selu')(sal)
sal=Dense(100,activation='selu')(sal)
sal=Dense(50,activation='relu')(sal)
sal = Dense(2, activation='softmax')(sal)
#Se unen la CNN y el top
efnetb0_custom=Model(inputs=model.input, outputs=sal)
efnetb0_custom.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])
history=efnetb0_custom.fit(x_train_res,y_train,batch_size=64,epochs=10, validation_data=(x_val_res,y_val),callbacks=[lr_reduce,earlystopper,checkpoint])

Epoch 1/10
38/38 [==============================] - 114s 3s/step - loss: 0.7817 - accuracy: 0.5121 - val_loss: 0.6955 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
38/38 [==============================] - 108s 3s/step - loss: 0.7432 - accuracy: 0.5217 - val_loss: 0.7003 - val_accuracy: 0.5000

Epoch 00002: val_accuracy did not improve from 0.50000
Epoch 3/10
38/38 [==============================] - 106s 3s/step - loss: 0.7194 - accuracy: 0.5454 - val_loss: 0.6995 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/10
38/38 [==============================] - 103s 3s/step - loss: 0.7187 - accuracy: 0.5354 - val_loss: 0.7034 - val_accuracy: 0.5000

Epoch 00004: val_accuracy did not improve from 0.50000
Epoch 5/10
38/38 [==============================] - 106s 3s/step - loss: 0.7089 - accuracy: 0.5279 - val_loss: 0.7149 - val_accuracy: 0.5000

Epoch 00005: val_accuracy did not improve from 0.50000
Epoch 6/10
38/38 [==============================] - 113s 3s/step - loss: 0.7175 - accuracy: 0.5312 - val_loss: 0.7100 - val_accuracy: 0.5000

Epoch 00006: val_accuracy did not improve from 0.50000
Epoch 7/10
38/3

In [54]:
pred=efnetb0_custom.predict(x_test_res)
pred=np.argmax(pred,axis=1)
y1=np.argmax(y_test,axis=1)

#label=np.argmax(yp_oh)
exactitud_test=0
for a in range(len(pred)):
    if pred[a]==y1[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')

exactitud de la prueba=  50.0 %


# InceptionV3

In [57]:
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint =  ModelCheckpoint('resnet50.h', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)    

x_train_res=tf.keras.applications.inception_v3.preprocess_input(np.copy(x_train))
x_val_res=tf.keras.applications.inception_v3.preprocess_input(np.copy(x_val))
x_test_res=tf.keras.applications.inception_v3.preprocess_input(np.copy(x_test))

model = tf.keras.applications.InceptionV3(input_shape=(x_size,y_size,3),weights=None,include_top=False) ## Colocar otro top
#model.summary()
sal=model.output
sal=Flatten()(sal)
sal=Dense(502,activation='relu')(sal)
sal=Dropout(0.26)(sal)
sal = Dense(256, activation='selu')(sal)
sal=Dense(100,activation='selu')(sal)
sal=Dense(50,activation='relu')(sal)
sal = Dense(2, activation='softmax')(sal)
#Se unen la CNN y el top
incv3_custom=Model(inputs=model.input, outputs=sal)
incv3_custom.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])
history=incv3_custom.fit(x_train_res,y_train,batch_size=64,epochs=10, validation_data=(x_val_res,y_val),callbacks=[lr_reduce,earlystopper,checkpoint])

Epoch 1/10
38/38 [==============================] - 66s 2s/step - loss: 0.7774 - accuracy: 0.4929 - val_loss: 0.6936 - val_accuracy: 0.4613

Epoch 00001: val_accuracy improved from -inf to 0.46125, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets
Epoch 2/10
38/38 [==============================] - 60s 2s/step - loss: 0.7294 - accuracy: 0.5183 - val_loss: 0.6953 - val_accuracy: 0.5000

Epoch 00002: val_accuracy improved from 0.46125 to 0.50000, saving model to resnet50.h
INFO:tensorflow:Assets written to: resnet50.h\assets
Epoch 3/10
38/38 [==============================] - 59s 2s/step - loss: 0.7275 - accuracy: 0.5083 - val_loss: 0.6953 - val_accuracy: 0.4712

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/10
38/38 [==============================] - 60s 2s/step - loss: 0.7169 - accuracy: 0.5146 - val_loss: 0.6920 - val_accuracy: 0.5125

Epoch 00004: val_accuracy improved from 0.50000 to 0.51250, saving model to resnet50.h
INFO:tensorflow:A

In [58]:
pred=incv3_custom.predict(x_test_res)
pred=np.argmax(pred,axis=1)
y1=np.argmax(y_test,axis=1)

#label=np.argmax(yp_oh)
exactitud_test=0
for a in range(len(pred)):
    if pred[a]==y1[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')

exactitud de la prueba=  47.5 %
